In [3]:
%pwd

'/home/acr/Documents/Projeto Sistemas Mecatrónicos/RL-SistMecatron'

In [4]:
%cd /home/acr/Documents/Projeto Sistemas Mecatrónicos/RL-SistMecatron

/home/acr/Documents/Projeto Sistemas Mecatrónicos/RL-SistMecatron


# Install Dependencies

In [5]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
# !pip install gym[atari]

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)
ERROR: No matching distribution found for tensorflow==2.3.0


# Creating a random environment

#### The goal is to balance the pole up, moving the cart to the left or right . For each step it takes a point, for a maximum of 200 points

In [6]:
import gym
import random
import time

In [7]:
env = gym.make('SpaceInvaders-v0') # To be able to get all the environments I had to clone the openAIGym repository and pip install -e.[all]
height, width, channels = env.observation_space.shape
actions = env.action_space.n

A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


In [8]:
env.unwrapped.get_action_meanings() # There are 6 possible actions that the spaceships can do

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [9]:
# help(env.compute_reward)

#### Now we're just setting some random episodes where there is no learning from each of the episodes

In [10]:
episodes = 10
for episode in range(1,episodes+1):
  state=env.reset()
  done=False
  score=0

  while not done:
    env.render() # Rendering of the screen
#     time.sleep(0.03) # Slowing the rendering. It stops 0.03 seconds for each frame
    action = random.choice([0,1,2,3,4,5]) # There are 6 possible actions
    n_state, reward, done, info = env.step(action)
    score+=reward
    sum_of_episodes +=score
  print('Episode:{} Score: {}'.format(episode, score))
print('Sum of all the random episodes:{}'.format(sum_of_episodes))
env.close()   # Closing the video, otherwise the window will not close

/home/acr/anaconda3/lib/python3.9/site-packages/gym/envs/atari/environment.py:255: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode:1 Score: 135.0
Episode:2 Score: 50.0
Episode:3 Score: 45.0
Episode:4 Score: 55.0
Episode:5 Score: 105.0
Episode:6 Score: 165.0
Episode:7 Score: 150.0
Episode:8 Score: 85.0
Episode:9 Score: 265.0
Episode:10 Score: 155.0


In [11]:
# help(env.step)

Help on method step in module gym.wrappers.time_limit:

step(action) method of gym.wrappers.time_limit.TimeLimit instance
    Run one timestep of the environment's dynamics. When end of
    episode is reached, you are responsible for calling `reset()`
    to reset this environment's state.
    
    Accepts an action and returns a tuple (observation, reward, done, info).
    
    Args:
        action (object): an action provided by the agent
    
    Returns:
        observation (object): agent's observation of the current environment
        reward (float) : amount of reward returned after previous action
        done (bool): whether the episode has ended, in which case further step() calls will return undefined results
        info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)



In [12]:
env.close()

# Create a Deep Learning Model with Keras

#### Ideally we want to take the score of each episode all the way up to 200. The deep RL is going to learn the best action to take in that specific environment to maximize the score

In [13]:
import numpy as np 
import tensorflow as tf  #allows a sequential model with Keras
# It's better to import tensorflow and use sequential than specifically importing the sequential from tensorlfow.keras.model
from tensorflow.keras.layers import Dense, Flatten, Convolution2D # We've imported the dense and flattten nodes. For the game we will give images of the game as input so a convolutional network will deal with the images
from tensorflow.keras.optimizers import Adam # We've imported the optimizer Adam to train the deep learning model

2021-12-07 19:16:41.393364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/acr/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2021-12-07 19:16:41.393400: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
def build_model(height, width, channels, actions):
    model =tf.keras.Sequential()  # initiating a sequential model
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels))) # As we will deal with images we start with a convolution network
    # It will have 32 convolution2D filters. These are the number of filters that detect the various elements of the game
    # The (8,8) is the size of the filters. In this case is 8 units by 8 units
    # The strides is how much the filter moves for each step, in this case it will be a considreable distance as it moves 4 to the side and 4 up or down. it is a diagonal move
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu')) # It passes through a flatten node which has the 4 different states
    model.add(Convolution2D(64, (3,3), activation='relu')) # It passes through a flatten node which has the 4 different states
    model.add(Flatten()) # Flatten layer. It takes all of our layers and flattens it into a single layer
    model.add(Dense(512, activation='relu')) # Dense layers are fully connected layers. Every single unit is connected to every unit of the next layer
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear')) # This makes sure that the output are the actions. First (or through the top), come the images and lastly comes the actions
    return model
# This model is fed the states and gets out the actions, and it trains in order to maximize the reward

In [31]:
del model_a # To be used only when it gets to the dqn= build_agent. For some reason it gives an error when we don't delete the model

NameError: name 'model_a' is not defined

In [34]:
model_a = build_model(height, width, channels, actions)

In [35]:
model_a.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_8 (Dense)             (None, 512)               34603520  
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 dense_10 (Dense)            (None, 6)                

# Build Agent with Keras-RL

In [36]:
from rl.agents import DQNAgent # Agent from keras rl, there are several
from rl.memory import SequentialMemory # For the DQNAgent there is the need for some memory, the Sequential memory allow it
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy # EpsGreedyQPolicy is what finds the best reward outcome. The linear annealed policy gives a decay. As we get clsoes to the optimal value, we start to close in


In [37]:
def build_agent(model, actions):
  policy_a = LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps', value_max=1.,value_min=.1,value_test=.2, nb_steps=10000)
  memory_a = SequentialMemory(limit=1000, window_length=3)
  dqn = DQNAgent(model=model_a, memory=memory_a, policy=policy_a, 
                 enable_dueling_network=True, dueling_type='avg',
                 nb_actions=actions, nb_steps_warmup=1000)
  return dqn
# Dueling networks split value and advantage, they help the model learn when to take action and when not to bother

In [38]:
# help(DQNAgent)

In [39]:
dqn = build_agent(model_a,actions) # Agent initiated. It knows the model, where it has the environment, and the actions it is allowed to do 
dqn.compile(Adam(learning_rate=1e-4)) # Compile the model through optimizer
dqn.fit(env, nb_steps=30000, visualize=False, verbose=2)
# As the deep learning model is huge it takes a long time to be trained and it is recommended 10 million steps to train it well. It can take a long long time, so we just train a few steps to see improvements but not the ultimate model
# The mean reward is what we want to maximize. It will bounce around for a while but will begin to converge the longer it is trained

Training for 30000 steps ...


/home/acr/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   650/30000: episode: 1, duration: 24.867s, episode steps: 650, steps per second:  26, episode reward: 155.000, mean reward:  0.238 [ 0.000, 30.000], mean action: 2.652 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  1330/30000: episode: 2, duration: 25.579s, episode steps: 680, steps per second:  27, episode reward: 210.000, mean reward:  0.309 [ 0.000, 30.000], mean action: 2.550 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  2145/30000: episode: 3, duration: 30.753s, episode steps: 815, steps per second:  27, episode reward: 225.000, mean reward:  0.276 [ 0.000, 30.000], mean action: 2.636 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  2782/30000: episode: 4, duration: 25.212s, episode steps: 637, steps per second:  25, episode reward: 135.000, mean reward:  0.212 [ 0.000, 30.000], mean action: 2.634 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  3787/30000: episode: 5, duration: 37.614s, episode steps: 1005, steps per second:  27, episode reward: 460.000

/home/acr/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  5197/30000: episode: 7, duration: 385.565s, episode steps: 757, steps per second:   2, episode reward: 125.000, mean reward:  0.165 [ 0.000, 30.000], mean action: 2.762 [0.000, 5.000],  loss: 1646.701584, mean_q: 18.371023, mean_eps: 0.541135
  6643/30000: episode: 8, duration: 2597.059s, episode steps: 1446, steps per second:   1, episode reward: 720.000, mean reward:  0.498 [ 0.000, 200.000], mean action: 2.405 [0.000, 5.000],  loss: 5.378485, mean_q: 10.535580, mean_eps: 0.467245
  7231/30000: episode: 9, duration: 1032.740s, episode steps: 588, steps per second:   1, episode reward: 55.000, mean reward:  0.094 [ 0.000, 20.000], mean action: 2.347 [0.000, 5.000],  loss: 1.879108, mean_q: 9.311108, mean_eps: 0.375715
  7622/30000: episode: 10, duration: 687.665s, episode steps: 391, steps per second:   1, episode reward: 60.000, mean reward:  0.153 [ 0.000, 25.000], mean action: 2.691 [0.000, 5.000],  loss: 0.713542, mean_q: 10.241191, mean_eps: 0.331660
  8554/30000: episode: 11, 

In [73]:
scores = dqn.test(env,nb_episodes=10, visualize=False)
print('Average reward:{}'.format(np.mean(scores.history['episode_reward'])))
print('Sum of the rewards:{}'.format(np.sum(scores.history['episode_reward'])))

Testing for 10 episodes ...
Episode 1: reward: 90.000, steps: 650
Episode 2: reward: 175.000, steps: 1346
Episode 3: reward: 80.000, steps: 637
Episode 4: reward: 280.000, steps: 1139
Episode 5: reward: 150.000, steps: 977
Episode 6: reward: 165.000, steps: 982
Episode 7: reward: 650.000, steps: 1700
Episode 8: reward: 290.000, steps: 985
Episode 9: reward: 165.000, steps: 865
Episode 10: reward: 440.000, steps: 1703
Average reward:248.5
Sum of the rewards:2485.0


In [81]:
scores.history

{'episode_reward': [120.0,
  25.0,
  280.0,
  230.0,
  220.0,
  385.0,
  470.0,
  310.0,
  190.0,
  155.0],
 'nb_steps': [474, 329, 933, 902, 615, 668, 1245, 712, 681, 826]}

# Reloading Agent from Memory

In [45]:
dqn.save_weights('SavedWeights/10k-3hours/dqn_weights.h5f',overwrite=True)

In [74]:
del model_a
del dqn
model_a = build_model(height, width, channels, actions)
dqn = build_agent(model_a,actions)
dqn.compile(Adam(learning_rate=1e-4))

In [75]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')

In [76]:
scores = dqn.test(env,nb_episodes=10, visualize=False)
print('Average reward:{}'.format(np.mean(scores.history['episode_reward'])))
print('Sum of the rewards:{}'.format(np.sum(scores.history['episode_reward'])))

Testing for 10 episodes ...
Episode 1: reward: 120.000, steps: 474
Episode 2: reward: 25.000, steps: 329
Episode 3: reward: 280.000, steps: 933
Episode 4: reward: 230.000, steps: 902
Episode 5: reward: 220.000, steps: 615
Episode 6: reward: 385.000, steps: 668
Episode 7: reward: 470.000, steps: 1245
Episode 8: reward: 310.000, steps: 712
Episode 9: reward: 190.000, steps: 681
Episode 10: reward: 155.000, steps: 826
Average reward:238.5
Sum of the rewards:2385.0


In [77]:
env.close()

In [78]:
%pwd

'/home/acr/Documents/Projeto Sistemas Mecatrónicos/RL-SistMecatron'